In [1]:
import numpy as np
import serial
import keyboard
import matplotlib.pyplot as plt

In [57]:
fpath = "F:\\pp1_data_processing\\2023-06-11 bluetooth data.txt"
with open(fpath, 'r') as f:
    print("adr\tbank\tval\t|\tadrbin\tbankbin\tvalbin\t\trawhex")
    print("---\t----\t---\t|\t------\t----\t------\t\t------")
    i = 0
    for line in f.readlines():
        linehex = int(line, base=16)
        bank = 'A' if (linehex & (1<<12))>0 else 'B'
        # OR with 0x8000 to make the bin string a fixed length
        # bit 15 is unused by firmware so this is fine for conversions
        adrbin = bin((linehex & int('0x0F00', base=16)) | int('0x8000', base=16))
        # reverse adrbin because it is in big endian
        adr = int(adrbin[9:5:-1], base=2)
        
        valbin = bin((linehex & int('0x00FF', base=16))| int('0x8000', base=16))
        # reverse val because it is in big endian
        val = int(valbin[-1:9:-1], base=2)
        print(f"{adr}\t{bank}\t{val}\t|\t{adrbin[9:5:-1]}\t{0 if bank=='A' else 1}\t{valbin[-1:9:-1]}\t{hex(linehex)}")
        i += 1


adr	bank	val	|	adrbin	bankbin	valbin		rawhex
---	----	---	|	------	----	------		------
8	B	255	|	1000	1	11111111	0x21ff
15	A	255	|	1111	0	11111111	0x1fff
7	B	64	|	0111	1	01000000	0x2e02
7	A	130	|	0111	0	10000010	0x1e41
0	B	20	|	0000	1	00010100	0x2028
5	A	255	|	0101	0	11111111	0x1aff
4	A	0	|	0100	0	00000000	0x1200
5	B	0	|	0101	1	00000000	0x2a00
13	B	247	|	1101	1	11110111	0x2bef
13	A	247	|	1101	0	11110111	0x1bef
0	B	0	|	0000	1	00000000	0x2000
0	A	0	|	0000	0	00000000	0x1000
8	A	63	|	1000	0	00111111	0x11fc
12	A	0	|	1100	0	00000000	0x1300
4	B	255	|	0100	1	11111111	0x22ff
4	A	255	|	0100	0	11111111	0x12ff
12	B	9	|	1100	1	00001001	0x2390
12	B	14	|	1100	1	00001110	0x2370
9	B	126	|	1001	1	01111110	0x297e
11	B	126	|	1011	1	01111110	0x2d7e
3	B	126	|	0011	1	01111110	0x2c7e
5	B	126	|	0101	1	01111110	0x2a7e
1	A	0	|	0001	0	00000000	0x1800
8	B	255	|	1000	1	11111111	0x21ff
9	A	255	|	1001	0	11111111	0x19ff
7	B	64	|	0111	1	01000000	0x2e02
7	A	130	|	0111	0	10000010	0x1e41
0	B	20	|	0000	1	00010100	0x2028
5	

In [2]:
memory_view = dict()
banks = ['A']*16 + ['B']*16
addresses = [x for x in range(16)] * 2
for adr,bank in zip(addresses, banks):
    memory_view[f'{adr}{bank}'] = 0
    print(f'{adr}{bank}:0')

0A:0
1A:0
2A:0
3A:0
4A:0
5A:0
6A:0
7A:0
8A:0
9A:0
10A:0
11A:0
12A:0
13A:0
14A:0
15A:0
0B:0
1B:0
2B:0
3B:0
4B:0
5B:0
6B:0
7B:0
8B:0
9B:0
10B:0
11B:0
12B:0
13B:0
14B:0
15B:0


In [3]:
def update_bar_plot(adr, bank, val, bars, fig):
    adrbnk = f'{adr}{bank}'
    i = list(memory_view.keys()).index(adrbnk)
    if i < 0:
        return
    bars[i].set_height(val)
    fig.canvas.draw()

In [41]:
prev_backend = 'module://matplotlib_inline.backend_inline'
plt.switch_backend('TkAgg')
%matplotlib tk


In [3]:

# need a way to ask user for their serial port after connecting to the ESP32 over bluetooth
# press esc to exit
port = 'COM3'
with serial.Serial(port, timeout=1) as ser:
    ser.flush()
    # fig, ax = plt.subplots()
    # ax.set_xlabel('Address & Bank')
    # ax.set_ylabel('Value')
    # ax.tick_params(axis='x', rotation=55)
    # bars = ax.bar(memory_view.keys(), memory_view.values())
    while True:
        if keyboard.is_pressed('esc'):
            break
        else:
            if ser.in_waiting > 0:
                line = ser.readline().decode()
                line = line.rstrip('\r\n')
                line = int(line, base=16)
                bank = 'A' if (line & (1<<12))>0 else 'B'
                adrbin = bin((line & int('0x0F00', base=16)) | int('0x8000', base=16))
                # reverse adrbin because it is in big endian
                adr = int(adrbin[9:5:-1], base=2)
                
                valbin = bin((line & int('0x00FF', base=16))| int('0x8000', base=16))
                # reverse val because it is in big endian
                val = int(valbin[-1:9:-1], base=2)
                
                # update the value in our memory_view dict
                memory_view[f'{adr}{bank}'] = val
                print(f'{adr}{bank},{val}')
                # update the visualizer
                # update_bar_plot(adr, bank, val, bars, fig)
                # plt.pause(0.000001)

# plt.show()

8B,255
7A,64
7A,130
0B,127
0A,127
5A,255
11B,0
11A,0
4B,0
4A,0
4B,0
13B,247
13A,247
13B,0
13A,0
0A,0
8B,63
8A,63
12B,0
12A,0
4B,255
4A,255
2B,255
2A,255
2B,25
12B,0
12A,0
9B,126
9A,126
11B,126
3B,126
5B,126
1B,0
1A,0
9B,126
9A,126
11B,126
3B,126
5B,126
1B,0
1A,0
8B,255
8A,255
7B,64
7A,64
7A,130
0B,127
0A,127
5A,255
11B,0
11A,0
4B,0
13B,247
13A,247
13B,0
13A,0
0A,0
8B,63
8A,63
12B,0
12A,0
4B,255
4A,255
2B,25
2A,25
2B,25
12B,0
12A,0
9B,126
9A,126
11B,126
3B,126
5B,126
1B,0
1A,0
8B,255
8A,255
7B,64
7A,64
0A,130
0B,20
5B,255
5A,255
11B,0
11A,0
4B,0
13B,247
13A,247
13B,0
13A,0
0A,0
8B,63
8A,63
12B,0
12A,0
4B,255
4A,255
2B,255
2A,25
2B,25
12B,0
12A,0
9B,126
9A,126
11B,126
3B,126
5B,126
1B,0
1A,0
8B,255
8A,255
7B,64
7A,64
0A,130
0B,20
5B,255
5A,255
11B,0
11A,0
4B,0
13B,247
13A,247
13B,0
13A,0
0A,0
8B,63
8A,63
12B,0
12A,0
4B,255
4A,255
3B,255
2A,25
2B,25
12B,0
12A,0
9B,126
9A,126
11B,126
3B,126
5B,126
1B,0
1A,0
8B,255
8A,255
7B,64
7A,64
0A,130
0B,20
5B,255
5A,255
11B,0
11A,0
4B,0
13B,247
13A,2

In [42]:
plt.switch_backend(prev_backend)
%matplotlib inline